In [1]:
import pandas as pd
import os
from pathlib import Path
import numpy as np
from datetime import datetime
import toolkitETL
import importlib
importlib.reload(toolkitETL)


<module 'toolkitETL' from 'd:\\Pedro\\Mestrado\\ETL-ML-Dengue\\src\\etl\\toolkitETL.py'>

In [2]:
## Estabelecendo diretórios base
diretorio_src_etl = os.getcwd()
diretorio_src = Path(diretorio_src_etl).parent.absolute()
diretorio_base = Path(diretorio_src).parent.absolute()
diretorio_base = "F:\\Mestrado_pedro"
diretorio_data = os.path.join(diretorio_base, 'data')

# Bronze
diretorio_bronze = os.path.join(diretorio_data, 'bronze')
diretorio_sim_bronze = os.path.join(diretorio_bronze, 'sim')
diretorio_sim_bronze_csv = os.path.join(diretorio_sim_bronze, 'csv')

# Silver
diretorio_silver = os.path.join(diretorio_data, 'silver')
diretorio_sim_silver = os.path.join(diretorio_silver, 'sim')
diretorio_sim_silver_tsv = os.path.join(diretorio_sim_silver, 'tsv')

## Criação de pastas para armazenar os arquivos
os.makedirs(os.path.join(diretorio_data), exist_ok=True)
os.makedirs(os.path.join(diretorio_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_sim_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_sim_silver_tsv), exist_ok=True)




In [14]:
arquivos_csv = [arquivo for arquivo in os.listdir(diretorio_sim_bronze_csv) if arquivo.endswith("csv")]
# arquivos_csv = arquivos_csv[-2:]
arquivos_tsv = [arquivo.rstrip("csv") + "tsv" for arquivo in arquivos_csv]
arquivos_tsv

['DOAC2010.tsv',
 'DOAC2011.tsv',
 'DOAC2012.tsv',
 'DOAC2013.tsv',
 'DOAC2014.tsv',
 'DOAC2015.tsv',
 'DOAC2016.tsv',
 'DOAC2017.tsv',
 'DOAC2018.tsv',
 'DOAC2019.tsv',
 'DOAC2020.tsv',
 'DOAC2021.tsv',
 'DOAL2010.tsv',
 'DOAL2011.tsv',
 'DOAL2012.tsv',
 'DOAL2013.tsv',
 'DOAL2014.tsv',
 'DOAL2015.tsv',
 'DOAL2016.tsv',
 'DOAL2017.tsv',
 'DOAL2018.tsv',
 'DOAL2019.tsv',
 'DOAL2020.tsv',
 'DOAL2021.tsv',
 'DOAM2010.tsv',
 'DOAM2011.tsv',
 'DOAM2012.tsv',
 'DOAM2013.tsv',
 'DOAM2014.tsv',
 'DOAM2015.tsv',
 'DOAM2016.tsv',
 'DOAM2017.tsv',
 'DOAM2018.tsv',
 'DOAM2019.tsv',
 'DOAM2020.tsv',
 'DOAM2021.tsv',
 'DOAP2010.tsv',
 'DOAP2011.tsv',
 'DOAP2012.tsv',
 'DOAP2013.tsv',
 'DOAP2014.tsv',
 'DOAP2015.tsv',
 'DOAP2016.tsv',
 'DOAP2017.tsv',
 'DOAP2018.tsv',
 'DOAP2019.tsv',
 'DOAP2020.tsv',
 'DOAP2021.tsv',
 'DOBA2010.tsv',
 'DOBA2011.tsv',
 'DOBA2012.tsv',
 'DOBA2013.tsv',
 'DOBA2014.tsv',
 'DOBA2015.tsv',
 'DOBA2016.tsv',
 'DOBA2017.tsv',
 'DOBA2018.tsv',
 'DOBA2019.tsv',
 'DOBA2020.tsv

In [6]:
# Função para converter NU_IDADE_N para idade em anos
def convert_to_years_sim(age):
    if pd.isnull(age):
        return np.nan
    if len(str(age)) < 3:
        return np.nan
    age_str = str(age)
    prefix = age_str[0]
    try:
        value = int(age_str[1:])
    except:
        return np.nan

    
    if prefix == '1':  # Minuto
        return value / (60 * 24 * 365)  # Aproximação para converter minutos em anos
    elif prefix == '2':  # Hora
        return value / (24 * 365)  # Aproximação para converter hora em anos
    elif prefix == '3':  # Mês
        return value / 12  # Aproximação para converter meses em anos
    elif prefix == '4':  # Ano
        return value
    elif prefix == '5':  # Ano
        return value + 100
    else:
        return np.nan



In [7]:
columns_to_convert = [
    "codmunres", "codmunocor", "idademae", "gestacao", "peso", "codigo_municipio", "nu_mes", "nu_ano"
]

colunas_data = [
    'DTINVESTIG', 'DTOBITO', 'DTNASC', 'DTCADASTRO', 'DTRECEBIM', 'DTCADINV', 'DTCONINV', 'DTCADINF', 'DTCONCASO', 'DTATESTADO'
]

colunas_importantes = [
    "tipobito", "dtobito", "dtnasc", "sexo", "racacor", "estciv", "esc", "codmunres", "lococor", "codmunocor", "idademae", "escmae", "gravidez", "gestacao", "parto", "obitoparto", "peso", "obitograv", "obitopuerp", "assistmed", "exame", "cirurgia", "necropsia", "linhaa", "linhab", "linhac", "linhad", "linhaii", "causabas", "dtatestado", "circobito", "acidtrab", "fonte", "tppos", "dtinvestig", "causabas_o", "dtcadastro", "fonteinv", "dtrecebim", "dtcadinf", "tpobitocor", "codigo_municipio", "idade_anos", "esc2010", "escmae2010", "stdoepidem", "stdonova", "nu_mes", "nu_ano"
]

colunas_decodificar = {
    'TIPOBITO': {
        1: "Fetal",
        2: "Não fetal"},
    'SEXO': {
        1: "Masculino",
        2: "Feminino",
        9: "Ignorado"
    },
    'RACACOR': {1: "Branca",
        2: "Preta",
        3: 'Amarela',
        4: 'Parda',
        5: 'Indígena'},
    'ESTCIV': {1: "Solteiro",
        2: "Casado",
        3: "Viúvo",
        4: "Separado",
        5: "União",
        9: "Ignorado"},
    'ESC': {
        1: "Nenhuma", 
        2: "De 1 a 3 anos", 
        3: "De 4 a 7 anos", 
        4: "De 8 a 11 anos", 
        5: "12 anos e mais", 
        9: "Ignorado"
    },
    'ESCMAE': {
        1: "Nenhuma", 
        2: "De 1 a 3 anos", 
        3: "De 4 a 7 anos", 
        4: "De 8 a 11 anos", 
        5: "12 anos e mais", 
        9: "Ignorado"
    },
    'ESC2010': {
        0: "Sem escolaridade",
        1: "Fundamental I (1ª a 4ª série)", 
        2: "Fundamental II (5ª a 8ª série)",
        3: "Médio (antigo 2º Grau)",
        4: "Superior incompleto",
        5: "Superior completo",
        9: "Ignorado"
    },
    'ESCMAE2010': {
        0: "Sem escolaridade",
        1: "Fundamental I (1ª a 4ª série)", 
        2: "Fundamental II (5ª a 8ª série)",
        3: "Médio (antigo 2º Grau)",
        4: "Superior incompleto",
        5: "Superior completo",
        9: "Ignorado"
    },
    'LOCOCOR': {
        1: "Hospital",
        2: "Outros estabelecimentos de saúde",
        3: "Domicílio",
        4: "Via pública",
        5: "Outros",
        6: "Aldeia indígena",
        9: "Ignorado"
    },
    'GRAVIDEZ': {
        1: "Única",
        2: "Dupla",
        3: "Tripla e mais",
        9: "Ignorada"
    },
    'PARTO': {
        1: "Vaginal",
        2: "Cesáreo",
        9: "Ignorado"
    },
    'OBITOPARTO': {
        1: "Antes",
        2: "Durante",
        3: "Depois",
        9: "Ignorado"
    },
    'OBITOGRAV': {
        1: "Sim",
        2: "Não",
        9: "Ignorado"
    },
    'OBITOPUERP': {
        1: "Sim, até 42 dias após o parto",
        2: "Sim, de 43 dias a 1 ano",
        3: "Não",
        9: "Ignorado"
    },
    'ASSISTMED': {
        1: "Sim",
        2: "Não",
        9: "Ignorado"
    },
    'EXAME': {
        1: "Sim",
        2: "Não",
        9: "Ignorado"
    },
    'CIRURGIA': {
        1: "Sim",
        2: "Não",
        9: "Ignorado"
    },
    'NECROPSIA': {
        1: "Sim",
        2: "Não",
        9: "Ignorado"
    },
    'CIRCOBITO': {
        1: "Acidente",
        2: "Suicídio",
        3: "Homicídio",
        4: "Outros",
        9: "Ignorado"
    },
    'ACIDTRAB': {
        1: "Sim",
        2: "Não",
        9: "Ignorado"
    },
    'FONTE': {
        1: "Ocorrência policial",
        2: "Hospital",
        3: "Família",
        4: "Outra",
        9: "Ignorado"
    },
    'FONTEINV': {
        1: "Comitê de Morte Materna e/ou Infantil",
        2: "Visita domiciliar / Entrevista família",
        3: "Estabelecimento de Saúde / Prontuário",
        4: "Relacionado com outros bancos de dados",
        5: "S V O",
        6: "I M L",
        7: "Outra fonte",
        8: "Múltiplas fontes",
        9: "Ignorado"
    },
    'STDOEPIDEM': {
        1: "Sim",
        0: "Não"
    },
    'STDONOVA': {
        1: "Sim",
        0: "Não"
    },
    'TPPOS': {
        1: "Sim",
        2: "Não"
    },
    'TPOBITOCOR': {
        1: "Durante a gestação",
        2: "Durante o abortamento",
        3: "Após o abortamento",
        4: "No parto ou até 1 hora após o parto",
        5: "No puerpério - até 42 dias após o parto",
        6: "Entre 43 dias e até 1 ano após o parto",
        7: "A investigação não identificou o momento do óbito",
        8: "Mais de um ano após o parto",
        9: "O óbito não ocorreu nas circunstâncias anteriores"
    }
}

def aplicar_decoficador(valor, coluna):
    if valor in colunas_decodificar[coluna].keys():
        return colunas_decodificar[coluna][valor]
    else:
        return np.nan

def formatar_datas(valor):
    if pd.isnull(valor):
        return np.nan
    try:
        return datetime.strptime(str(valor), "%d%m%Y").date()
    except ValueError:
        return np.nan

# Função auxiliar para extrair o mês
def extract_month(date):
    if pd.isnull(date):
        return np.nan
    return date.month

# Função auxiliar para extrair o ano
def extract_year(date):
    if pd.isnull(date):
        return np.nan
    return date.year

In [15]:
# Checando qual foi o último id adicionado
num_linhas_total = 0

for arquivo_tsv in arquivos_tsv:
    diretorio_tsv = os.path.join(diretorio_sim_silver_tsv, arquivo_tsv)
    if os.path.exists(diretorio_tsv):
        df = pd.read_csv(diretorio_tsv, sep='\t')
        ultimo_id = df['sim_id'].max()
        if ultimo_id > num_linhas_total:
            num_linhas_total = ultimo_id
num_linhas_total += 1
print(num_linhas_total)



1


In [ ]:
for arquivo_csv, arquivo_tsv in zip(arquivos_csv, arquivos_tsv):
    diretorio_tsv = os.path.join(diretorio_sim_silver_tsv, arquivo_tsv)
    if os.path.exists(diretorio_tsv):
        continue
    
    df = pd.read_csv(os.path.join(diretorio_sim_bronze_csv, arquivo_csv), sep=',', encoding="latin1")
    df['Codigo_Municipio'] = df['CODMUNOCOR'].apply(lambda x: str(x)[:-1] if len (str(x)) == 7 else str(x))
    df['IDADE_ANOS'] = df['IDADE'].apply(convert_to_years_sim)
    for coluna in colunas_decodificar.keys():
        if coluna not in df.columns:
            df[coluna] = np.nan
            continue
        df[coluna] = df[coluna].apply(lambda x: aplicar_decoficador(x, coluna))
    
    for coluna in colunas_data:
        if coluna not in df.columns:
            df[coluna] = np.nan
            continue
        df[coluna] = df[coluna].apply(formatar_datas)
    
    df['NU_MES'] = df['DTOBITO'].apply(extract_month)
    df['NU_ANO'] = df['DTOBITO'].apply(extract_year)

    df.columns = df.columns.str.lower()

    df.replace("NA", np.nan, inplace=True)
    df.replace("", np.nan, inplace=True)
    df = df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)
    df = df[colunas_importantes]


    # # # Itera sobre cada coluna e tenta converter para inteiro
    for column in columns_to_convert:
        df[column] = pd.to_numeric(df[column], errors='coerce').astype('Int64')
    
    colunas_para_verificar = ["codigo_municipio", "nu_mes", "nu_ano"]

    # Remover linhas que tenham valores nulos em qualquer uma dessas colunas
    df = df.dropna(subset=colunas_para_verificar)

    num_linhas_atual = len(df)
    df["sim_id"] = range(num_linhas_total, (num_linhas_total + num_linhas_atual))
    
    num_linhas_total += num_linhas_atual
    df.to_csv(os.path.join(diretorio_sim_silver_tsv, arquivo_tsv), sep='\t', index=False)

In [ ]:
arquivos_tsv = os.listdir(diretorio_sim_silver_tsv)
arquivos_tsv

In [4]:
diretorio_pos_banco = os.path.join(diretorio_sim_silver, "banco")
if not os.path.exists(diretorio_pos_banco): 
    os.makedirs(diretorio_pos_banco)


diretorio_dotenv = os.path.join(diretorio_base, ".env")

cursor, conn = toolkitETL.conectar_banco(diretorio_dotenv)
for arquivo_tsv in arquivos_tsv:
    
    caminho_arquivo = os.path.join(diretorio_sim_silver_tsv, arquivo_tsv)
    toolkitETL.inserir_no_banco(cursor, conn, caminho_arquivo, 'sim', delimiter='\t', diretorio_dotenv=diretorio_dotenv)

    caminho_arquivo_pos_banco = os.path.join(diretorio_pos_banco, arquivo_tsv)
    os.rename(caminho_arquivo, caminho_arquivo_pos_banco)